In [1]:
import pandas as pd
import torch
import numpy as np
import os
import re
from transformers import BertTokenizer, BertModel, BertForMaskedLM, pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import OneHotEncoder
from transformers import DataCollatorForLanguageModeling
import joblib
import pickle
from sklearn.metrics import silhouette_score
from torch import nn
from datasets import Dataset
from transformers import Trainer, TrainingArguments

# Set directories for saving models and loading datasets
model_dir = '/Users/n7/Desktop/ie University SAMBD Acadamics/Capstone Project Revised/Machine Learning models/Trained Models/Version7.7'
data_path = '/Users/n7/Desktop/ie University SAMBD Acadamics/Capstone Project Revised/Data/MINDlarge_test/Cleaned Dataset/News_cleaned.csv'
output_data_dir = '/Users/n7/Desktop/ie University SAMBD Acadamics/Capstone Project Revised/Data/MINDlarge_test/Cleaned Dataset'
output_data_path = os.path.join(output_data_dir, 'News_Features_Engineered_Dataset.parquet')
embedding_output_path = os.path.join(output_data_dir, 'news_embeddings_BERT_and_Kmeans.npy')

# Ensure model directory exists
os.makedirs(model_dir, exist_ok=True)

# Allowing for parallelism
os.environ["TOKENIZERS_PARALLELISM"] = "true"

# Set the device to MPS if available, otherwise use CPU
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# Helper function to standardize News ID format
def standardize_news_id(nid):
    """Standardize News ID by removing extra spaces."""
    return re.sub(r"\s+", "", nid) if isinstance(nid, str) else nid

# Load the dataset
print("Loading dataset...")
data = pd.read_csv(data_path)
print("Dataset loaded.")

# Standardize News ID
print("Standardizing News IDs...")
data['News ID'] = data['News ID'].apply(standardize_news_id)

# Combine text columns into a single representation for each article
print("Combining text columns...")
data["text"] = (
    data["Category"] + " " +
    data["Subcategory"] + " " +
    data["Title"] + " " +
    data["Abstract"]
)

# Fine-Tune BERT for MLM
def fine_tune_bert_for_mlm(data, model_dir):
    """
    Fine-tune BERT for Masked Language Modeling (MLM).
    Args:
        data (pd.DataFrame): Input data containing a "text" column.
        model_dir (str): Directory to save tokenizer and model.
    """
    print("Step 1: Fine-tuning BERT for Masked Language Modeling...")
    
    # Convert dataset to Hugging Face Dataset format
    print("Converting dataset...")
    dataset = Dataset.from_pandas(data[["text"]])

    # Load or train tokenizer
    tokenizer_path = os.path.join(model_dir, "bert_tokenizer")
    if os.path.exists(tokenizer_path):
        print("Loading existing BERT tokenizer...")
        tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
    else:
        print("Training new BERT tokenizer...")
        tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        tokenizer.save_pretrained(tokenizer_path)
        print(f"BERT tokenizer trained and saved to {tokenizer_path}")

    # Tokenize dataset
    print("Tokenizing dataset...")
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
    print("Tokenization completed.")

    # Data collator for MLM
    print("Initializing data collator for MLM...")
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

    # Load or use existing model
    model_path = os.path.join(model_dir, "bert_mlm_model")
    if os.path.exists(model_path):
        print("Loading existing BERT model for Masked Language Modeling...")
        model = BertForMaskedLM.from_pretrained(model_path)
    else:
        print("Training new BERT model for Masked Language Modeling...")
        model = BertForMaskedLM.from_pretrained("bert-base-uncased")

        # Training arguments
        print("Setting up training arguments...")
        training_args = TrainingArguments(
            output_dir=model_dir,
            eval_strategy="no",
            learning_rate=5e-5,
            per_device_train_batch_size=16,
            num_train_epochs=3,
            save_steps=10_000,
            save_total_limit=2,
            logging_dir="./logs",
            logging_steps=500
        )

        # Trainer
        print("Initializing Trainer...")
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_dataset,
            data_collator=data_collator,
            tokenizer=tokenizer
        )

        # Train the model
        print("Starting training...")
        trainer.train()
        print("Saving trained BERT model...")
        model.save_pretrained(model_path)
        print(f"BERT model trained and saved to {model_path}")

    print("BERT fine-tuning completed successfully.")
    
fine_tune_bert_for_mlm(data, model_dir)

# Generate Embeddings
def generate_embeddings(data, model_dir, output_data_dir, device, output_dim=768):
    """
    Generate 768-dimensional embeddings for text using a fine-tuned BERT model.
    Args:
        data (pd.DataFrame): Input data containing a "text" column.
        model_dir (str): Directory containing the fine-tuned model.
        output_data_dir (str): Directory to save the generated embeddings.
        device (torch.device): Device to run the model on.
        output_dim (int): Dimension of the output embeddings.
    """
    print("Step 2: Generating embeddings...")
    
    # Load BERT model and tokenizer
    bert_model_path = os.path.join(model_dir, "bert_mlm_model")
    print(f"Loading BERT model from {bert_model_path}...")
    bert_model = BertModel.from_pretrained(bert_model_path).to(device)
    print("BERT model loaded successfully.")

    tokenizer_path = os.path.join(model_dir, "bert_tokenizer")
    print(f"Loading tokenizer from {tokenizer_path}...")
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
    print("Tokenizer loaded successfully.")

    # Batch processing for embedding generation
    def generate_embeddings_batch(texts, model, tokenizer, device, batch_size=32):
        model.eval()
        embeddings = []
        print("Generating embeddings in batches...")
        with torch.no_grad():
            for i in range(0, len(texts), batch_size):
                batch_texts = texts[i:i + batch_size]
                inputs = tokenizer(
                    batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512
                ).to(device)
                outputs = model(**inputs)
                embeddings.append(outputs.pooler_output.cpu().numpy())
                print(f"Processed batch {i // batch_size + 1} of {len(texts) // batch_size + 1}")
        return np.vstack(embeddings)

    print("Generating embeddings for the dataset...")
    embeddings = generate_embeddings_batch(data["text"].tolist(), bert_model, tokenizer, device)
    print("Embeddings generation completed.")

    # Save embeddings
    print("Saving embeddings...")
    data["BERT-Embeddings"] = embeddings.tolist()
    np.save(os.path.join(output_data_dir, "news_embeddings.npy"), embeddings)
    data[["News ID", "BERT-Embeddings"]].to_parquet(os.path.join(output_data_dir, "news_BERT_Only_embeddings.parquet"), index=False)
    print(f"Embeddings saved to {output_data_dir} successfully.")

generate_embeddings(data, model_dir, output_data_dir, device, output_dim=768)

# Step 3: Sentiment and Emotion Analysis
sentiment_model_dir = os.path.join(model_dir, "sentiment_model")
if not os.path.exists(sentiment_model_dir):
    print("Training and saving sentiment model...")
    sentiment_pipeline = pipeline(
        "sentiment-analysis",
        model="distilbert/distilbert-base-uncased-finetuned-sst-2-english",
        revision="714eb0f",
        device=0 if torch.backends.mps.is_available() else -1
    )
    sentiment_pipeline.model.save_pretrained(sentiment_model_dir)
    sentiment_pipeline.tokenizer.save_pretrained(sentiment_model_dir)
    print("Sentiment model trained and saved.")
else:
    print("Loading existing sentiment model...")
    sentiment_pipeline = pipeline(
        "sentiment-analysis",
        model=sentiment_model_dir,
        device=0 if torch.backends.mps.is_available() else -1
    )

data['Sentiment'] = data.apply(
    lambda row: sentiment_pipeline(row['Title'][:512] + " " + row['Abstract'][:512])[0]['label'], axis=1
)

emotion_model_dir = os.path.join(model_dir, "emotion_model")
if not os.path.exists(emotion_model_dir):
    print("Training and saving emotion model...")
    emotion_pipeline = pipeline(
        "text-classification",
        model="bhadresh-savani/distilbert-base-uncased-emotion",
        device=0 if torch.backends.mps.is_available() else -1
    )
    emotion_pipeline.model.save_pretrained(emotion_model_dir)
    emotion_pipeline.tokenizer.save_pretrained(emotion_model_dir)
    print("Emotion model trained and saved.")
else:
    print("Loading existing emotion model...")
    emotion_pipeline = pipeline(
        "text-classification",
        model=emotion_model_dir,
        device=0 if torch.backends.mps.is_available() else -1
    )

data['Emotion'] = data.apply(
    lambda row: emotion_pipeline(row['Title'][:512] + " " + row['Abstract'][:512])[0]['label'], axis=1
)

# Step 4: One-Hot Encoding and K-means Clustering
encoder_path = os.path.join(model_dir, 'one_hot_encoder.pkl')

# Combine features as they are
combined_features = np.column_stack((
    data['Category'].astype(str), 
    data['Subcategory'].astype(str), 
    data['Sentiment'].astype(str), 
    data['Emotion'].astype(str)
))

# Train or load the one-hot encoder
if os.path.exists(encoder_path):
    print("Loading existing OneHotEncoder...")
    encoder = joblib.load(encoder_path)
else:
    print("Training new OneHotEncoder...")
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoder.fit(combined_features)
    joblib.dump(encoder, encoder_path)
    print(f"OneHotEncoder trained and saved to {encoder_path}")

# Apply one-hot encoding
print("Applying one-hot encoding to features...")
one_hot_encoded_features = encoder.transform(combined_features)

# Proceed with K-means clustering
kmeans_path = os.path.join(model_dir, 'kmeans_model.pkl')
if os.path.exists(kmeans_path):
    print("Loading existing K-means model...")
    kmeans = joblib.load(kmeans_path)
else:
    print("Training K-means model...")
    max_k = 100
    best_k = 2
    best_score = -1
    for k in range(2, max_k + 1):
        kmeans_candidate = MiniBatchKMeans(n_clusters=k, batch_size=1000, random_state=0)
        cluster_labels = kmeans_candidate.fit_predict(one_hot_encoded_features)
        score = silhouette_score(one_hot_encoded_features, cluster_labels)
        if score > best_score:
            best_k = k
            best_score = score
    kmeans = MiniBatchKMeans(n_clusters=best_k, batch_size=1000, random_state=0)
    kmeans.fit(one_hot_encoded_features)
    joblib.dump(kmeans, kmeans_path)
    print("K-means model trained and saved.")

# Assign clusters to data
data['K-means Clusters'] = kmeans.predict(one_hot_encoded_features)
centroids = kmeans.cluster_centers_

# Add K-means embeddings to data
data['K-means Embeddings'] = data['K-means Clusters'].apply(lambda x: centroids[x].tolist())

# Save final data and embeddings
data[['News ID', 'BERT-Embeddings', 'Sentiment', 'Emotion', 'K-means Clusters', 'K-means Embeddings']].to_parquet(output_data_path, index=False)
news_embeddings = {
    "News ID": data['News ID'].tolist(),
    "BERT-Embeddings": np.array(data['BERT-Embeddings'].tolist()),
    "K-means Embeddings": np.array(data['K-means Embeddings'].tolist())
}
np.save(embedding_output_path, news_embeddings)

print(f"Data and embeddings saved to {output_data_dir}")

Loading dataset...
Dataset loaded.
Standardizing News IDs...
Combining text columns...
Step 1: Fine-tuning BERT for Masked Language Modeling...
Converting dataset...
Loading existing BERT tokenizer...
Tokenizing dataset...


Map:   0%|          | 0/114247 [00:00<?, ? examples/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Tokenization completed.
Initializing data collator for MLM...
Loading existing BERT model for Masked Language Modeling...


Some weights of BertModel were not initialized from the model checkpoint at /Users/n7/Desktop/ie University SAMBD Acadamics/Capstone Project Revised/Machine Learning models/Trained Models/Version7.7/bert_mlm_model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT fine-tuning completed successfully.
Step 2: Generating embeddings...
Loading BERT model from /Users/n7/Desktop/ie University SAMBD Acadamics/Capstone Project Revised/Machine Learning models/Trained Models/Version7.7/bert_mlm_model...
BERT model loaded successfully.
Loading tokenizer from /Users/n7/Desktop/ie University SAMBD Acadamics/Capstone Project Revised/Machine Learning models/Trained Models/Version7.7/bert_tokenizer...
Tokenizer loaded successfully.
Generating embeddings for the dataset...
Generating embeddings in batches...
Processed batch 1 of 3571
Processed batch 2 of 3571
Processed batch 3 of 3571
Processed batch 4 of 3571
Processed batch 5 of 3571
Processed batch 6 of 3571
Processed batch 7 of 3571
Processed batch 8 of 3571
Processed batch 9 of 3571
Processed batch 10 of 3571
Processed batch 11 of 3571
Processed batch 12 of 3571
Processed batch 13 of 3571
Processed batch 14 of 3571
Processed batch 15 of 3571
Processed batch 16 of 3571
Processed batch 17 of 3571
Process

In [2]:
import pandas as pd
import re

# Load the behavior dataset
behavior_data_path = '/Users/n7/Desktop/ie University SAMBD Acadamics/Capstone Project Revised/Data/MINDlarge_test/Cleaned Dataset/cleaned_behavior_dataset.csv'
behavior_data = pd.read_csv(behavior_data_path)

# Columns to retain
columns_to_keep = ['User ID', 'Displayed News List', 'Clicked News IDs', 'Not-Clicked News IDs']

# Drop all columns except the ones specified
behavior_data = behavior_data[columns_to_keep]

# Check and display the data types of each column before processing
print("Data types before conversion:\n", behavior_data.dtypes)

# Function to convert to comma-separated and handle concatenated news IDs
def convert_to_comma_separated(value):
    if pd.isna(value):
        return ""  # Handle NaN values
    # Remove brackets, quotes, spaces, and any unwanted characters
    value = re.sub(r"[\[\]'\" ]", "", value)
    # Add comma between concatenated IDs using regex to find patterns like 'N[digits]N[digits]'
    value = re.sub(r"(N\d+)(?=N\d+)", r"\1,", value)
    return value

# Apply conversion to 'Displayed News List', 'Clicked News IDs', and 'Not-Clicked News IDs'
behavior_data['Displayed News List'] = behavior_data['Displayed News List'].apply(convert_to_comma_separated)
behavior_data['Clicked News IDs'] = behavior_data['Clicked News IDs'].apply(convert_to_comma_separated)
behavior_data['Not-Clicked News IDs'] = behavior_data['Not-Clicked News IDs'].apply(convert_to_comma_separated)

# Validation function to confirm no concatenated IDs remain
def check_concatenated_ids(column):
    pattern = r"N\d+N\d+"  # Pattern to detect concatenated IDs
    concatenated_ids = behavior_data[column].str.contains(pattern, regex=True).sum()
    return concatenated_ids == 0

# Check for concatenated IDs in all relevant columns
columns_to_check = ['Displayed News List', 'Clicked News IDs', 'Not-Clicked News IDs']
for col in columns_to_check:
    if check_concatenated_ids(col):
        print(f"No concatenated IDs found in '{col}'.")
    else:
        print(f"Concatenated IDs detected in '{col}' after processing.")

# Ensure 'Clicked News IDs' and 'Not-Clicked News IDs' have the same data type as 'Displayed News List'
displayed_news_type = behavior_data['Displayed News List'].dtype
behavior_data['Clicked News IDs'] = behavior_data['Clicked News IDs'].astype(displayed_news_type)
behavior_data['Not-Clicked News IDs'] = behavior_data['Not-Clicked News IDs'].astype(displayed_news_type)

# Check and display the data types after conversion to confirm changes
print("Data types after conversion:\n", behavior_data.dtypes)

# Display the first few rows to confirm the dataset structure
print("Processed Behavior Dataset:\n", behavior_data.head(10))

# Save the processed dataset if needed
output_behavior_data_path = '/Users/n7/Desktop/ie University SAMBD Acadamics/Capstone Project Revised/Data/MINDlarge_test/Cleaned Dataset/processed_behavior_dataset.csv'
behavior_data.to_csv(output_behavior_data_path, index=False)

Data types before conversion:
 User ID                  object
Displayed News List      object
Clicked News IDs        float64
Not-Clicked News IDs    float64
dtype: object
No concatenated IDs found in 'Displayed News List'.
No concatenated IDs found in 'Clicked News IDs'.
No concatenated IDs found in 'Not-Clicked News IDs'.
Data types after conversion:
 User ID                 object
Displayed News List     object
Clicked News IDs        object
Not-Clicked News IDs    object
dtype: object
Processed Behavior Dataset:
    User ID                                Displayed News List  \
0   U64099  N121133,N104200,N43255,N55860,N128965,N38014,N...   
1  U231077  N45124,N84730,N45128,N104312,N70022,N99111,N26...   
2  U606012  N59893,N84662,N90686,N33265,N127225,N120859,N6...   
3  U320649  N110863,N7889,N86335,N85056,N115743,N63372,N19...   
4  U357840  N98596,N85005,N15713,N67779,N47961,N55571,N666...   
5  U524079  N123633,N41661,N61319,N104200,N121104,N795,N12...   
6  U593249  N130269,N

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import os
import pickle
from ast import literal_eval
import re
import torch.optim as optim
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from collections import defaultdict

# Directory paths
model_dir = '/Users/n7/Desktop/ie University SAMBD Acadamics/Capstone Project Revised/Machine Learning models/Trained Models/Version7.7'
test_data_path = '/Users/n7/Desktop/ie University SAMBD Acadamics/Capstone Project Revised/Data/MINDlarge_test/Cleaned Dataset/News_Features_Engineered_Dataset.parquet'
behavior_test_path = '/Users/n7/Desktop/ie University SAMBD Acadamics/Capstone Project Revised/Data/MINDlarge_test/Cleaned Dataset/processed_behavior_dataset.csv'
user_profiles_path = os.path.join(model_dir, 'user_profiles.pkl')

# Device setup
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# Load user profiles
print("Loading saved user profiles...")
with open(user_profiles_path, 'rb') as f:
    user_profiles = pickle.load(f)

# Generate default profile for unseen users
print("Calculating default profile for unseen users...")
all_preferences = [profile['preference_profile'] for profile in user_profiles.values()]
all_non_preferences = [profile['non_preference_profile'] for profile in user_profiles.values()]

default_preference = torch.stack(all_preferences).mean(dim=0)
default_non_preference = torch.stack(all_non_preferences).mean(dim=0)
default_user_profile = {'preference_profile': default_preference, 'non_preference_profile': default_non_preference}

# Helper function to standardize News ID format
def standardize_news_id(nid):
    """Standardize News ID by removing extra spaces."""
    return re.sub(r"\s+", "", nid) if isinstance(nid, str) else nid

# Helper function to standardize User ID format
def standardize_user_id(uid):
    """Standardize User ID by removing extra spaces and applying a consistent format."""
    return re.sub(r"\s+", "", str(uid)) if isinstance(uid, (str, int)) else uid

# Step 1: Load news embeddings
print("Step 1: Loading news embeddings.")

def load_parquet_embeddings_with_error_checking(test_data_path):
    """
    Load embeddings from a Parquet file, validate them, and count any errors during processing.

    Args:
        test_data_path (str): Path to the Parquet file containing saved embeddings.

    Returns:
        pd.DataFrame: DataFrame containing News ID, BERT-Embeddings, and K-means Embeddings.
    """
    # Initialize error count
    error_count = 0

    # Load the Parquet file
    try:
        embeddings_df = pd.read_parquet(test_data_path)
    except FileNotFoundError:
        raise FileNotFoundError(f"The file {test_data_path} does not exist.")
    except Exception as e:
        raise RuntimeError(f"Failed to load Parquet file: {e}")

    # Check for required columns
    required_columns = ["News ID", "BERT-Embeddings", "K-means Embeddings"]
    for column in required_columns:
        if column not in embeddings_df.columns:
            raise KeyError(f"Missing expected column in the Parquet file: {column}")

    # Helper function to validate and convert embeddings to float32
    def validate_and_convert_embedding(embedding, expected_size):
        nonlocal error_count
        try:
            array = np.array(embedding, dtype=np.float32)  # Convert to float32
            if array.size != expected_size:
                raise ValueError(f"Unexpected size: {array.size}")
            return array
        except (ValueError, TypeError):
            error_count += 1
            return np.zeros(expected_size, dtype=np.float32)  # Default to zero array on error

    # Expected sizes
    bert_embeddings = embeddings_df["BERT-Embeddings"].iloc[0]
    kmeans_embeddings = embeddings_df["K-means Embeddings"].iloc[0]
    bert_size = len(bert_embeddings) if isinstance(bert_embeddings, (list, np.ndarray)) else 768
    kmeans_size = len(kmeans_embeddings) if isinstance(kmeans_embeddings, (list, np.ndarray)) else 304

    # Validate embeddings and convert to float32
    embeddings_df["BERT-Embeddings"] = embeddings_df["BERT-Embeddings"].apply(lambda x: validate_and_convert_embedding(x, bert_size))
    embeddings_df["K-means Embeddings"] = embeddings_df["K-means Embeddings"].apply(lambda x: validate_and_convert_embedding(x, kmeans_size))

    # Print the error count
    print(f"Number of errors encountered during embedding validation: {error_count}")
    return embeddings_df

# Load the embeddings
data = load_parquet_embeddings_with_error_checking(test_data_path)

# Standardize News IDs
data['News ID'] = data['News ID'].apply(standardize_news_id)

# Inspect the DataFrame
print(data.head())

# Concatenate BERT-Embeddings (768 dimensions) and K-means Embeddings (304 dimensions)
bert_embedding_size = len(data['BERT-Embeddings'].iloc[0])  # Infer size from the first embedding
kmeans_embedding_size = len(data['K-means Embeddings'].iloc[0])  # Infer size from the first embedding

# Validate embedding dimensions before concatenation
print(f"BERT Embedding Size: {bert_embedding_size}")
print(f"K-means Embedding Size: {kmeans_embedding_size}")

# Ensure all embeddings are of expected dimensions
assert all(len(embedding) == bert_embedding_size for embedding in data['BERT-Embeddings']), \
    "Inconsistent BERT embedding size detected."
assert all(len(embedding) == kmeans_embedding_size for embedding in data['K-means Embeddings']), \
    "Inconsistent K-means embedding size detected."

def pad_embeddings(matrix, target_size=1088):
    """
    Pad the concatenated embeddings matrix to a fixed size along the second dimension.
    Adds zero-padding to the end of each row if the size is less than the target size (default: 1088).

    Parameters:
    - matrix (np.ndarray): Input concatenated embeddings matrix.
    - target_size (int, optional): Desired size along the second dimension (default: 1088).

    Returns:
    - np.ndarray: Padded matrix with dimensions [matrix.shape[0], target_size].
    """
    current_size = matrix.shape[1]
    if current_size < target_size:
        # Calculate the padding size
        padding_size = target_size - current_size
        # Apply zero-padding along the second dimension
        padded_matrix = np.pad(matrix, ((0, 0), (0, padding_size)), mode='constant', constant_values=0)
        print(f"Padded embeddings from size {current_size} to {target_size}.")
    else:
        padded_matrix = matrix  # No padding needed if already at or above target size
        print(f"No padding applied. Embeddings already of size {current_size}.")
    return padded_matrix

# Step 2: Concatenate embeddings and news embeddings map generation
print("Step 2: Concatenate embeddings and news embeddings map generation.")
embedding_matrix = np.hstack([
    np.vstack(data['BERT-Embeddings'].values),  # Convert BERT-Embeddings to matrix form
    np.vstack(data['K-means Embeddings'].values)  # Convert K-means Embeddings to matrix form
])

# Pad concatenated embeddings to a fixed size (e.g., 1088 dimensions)
target_size = 1088
padded_embedding_matrix = pad_embeddings(embedding_matrix, target_size)

# Convert to tensors
embedding_tensor = torch.tensor(padded_embedding_matrix, dtype=torch.float32)

# Add the tensor embeddings to the dataset
data['combined_embedding'] = [embedding_tensor[i] for i in range(embedding_tensor.size(0))]

# Create news embeddings map
news_embeddings_map = {row['News ID']: row['combined_embedding'] for _, row in data.iterrows()}

# Print information for verification
print(f"Shape of concatenated embeddings: {embedding_matrix.shape}")
print(f"Shape of padded embeddings: {padded_embedding_matrix.shape}")
print(f"Sample news embeddings map: {list(news_embeddings_map.items())[:5]}")

# Step 3: Load behavior test data
print("Step 3: Loading behavior test data...")
behavior_df = pd.read_csv(behavior_test_path)

# Standardize User IDs
behavior_df['User ID'] = behavior_df['User ID'].apply(standardize_user_id)

# Standardize 'Displayed News List'
behavior_df['Displayed News List'] = behavior_df['Displayed News List'].apply(
    lambda x: [standardize_news_id(nid) for nid in x.split(',')] if isinstance(x, str) else []
)

# Set display options to avoid truncation
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

# Print first 5 rows for verification
print("First 5 rows after processing:")
print(behavior_df[['User ID', 'Displayed News List', 'Clicked News IDs', 'Not-Clicked News IDs']].head())

# Print size of test data after filtering
print(f"Test data size after filtering: {len(behavior_df)} rows")

Using device: mps
Loading saved user profiles...
Calculating default profile for unseen users...
Step 1: Loading news embeddings.
Number of errors encountered during embedding validation: 0
  News ID                                    BERT-Embeddings Sentiment  \
0  N88753  [-0.49939227, -0.010149627, 0.38407016, -0.373...  POSITIVE   
1  N45436  [-0.4372922, 0.29049647, 0.3156791, -0.1229524...  POSITIVE   
2  N23144  [-0.25999886, 0.36037475, 0.45065236, -0.32535...  NEGATIVE   
3  N93187  [-0.40450674, 0.1736807, 0.29816663, -0.173287...  NEGATIVE   
4  N75236  [-0.32645518, 0.097008444, 0.28911364, -0.1914...  NEGATIVE   

   Emotion  K-means Clusters  \
0    anger                10   
1    anger                10   
2      joy                66   
3     fear                37   
4  sadness                72   

                                  K-means Embeddings  
0  [0.05628373, 0.00539707, 0.07555898, 0.0485736...  
1  [0.05628373, 0.00539707, 0.07555898, 0.0485736...  
2  [0.0

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import pandas as pd

# File path to save processed dataset
output_csv_path = '/Users/n7/Desktop/ie University SAMBD Acadamics/Capstone Project Revised/Data/MINDlarge_test/Behavior Predictions Test Results/behavior_predictions_results.csv'

# Hyperparameters
BATCH_SIZE = 10000  # Adjustable batch size for performance tuning

# Initialize count tracking
count_tracker = {"news_ids_found": 0, "news_ids_not_found": 0}

# Step 1: Load Testing Data
print("Testing data loaded successfully during the data processing steps.")

# Step 2: Helper Functions
def calculate_cosine_similarity(vec1, vec2):
    """Calculate cosine similarity between two vectors."""
    return cosine_similarity(vec1.reshape(1, -1), vec2.reshape(1, -1))[0][0]

def classify_news_parallel(news_id, preference_profile, non_preference_profile, count_tracker):
    """
    Classify a single news ID as clicked or not-clicked based on scores.
    """
    news_embedding = news_embeddings_map.get(news_id)
    if news_embedding is None:
        count_tracker["news_ids_not_found"] += 1
        return None, None  # News ID not found

    count_tracker["news_ids_found"] += 1

    # Calculate scores
    preference_score = calculate_cosine_similarity(preference_profile, news_embedding)
    non_preference_score = calculate_cosine_similarity(non_preference_profile, news_embedding)

    # Classify based on scores
    if non_preference_score > preference_score:
        return news_id, f"{news_id}-0"  # Not-clicked
    else:
        return news_id, f"{news_id}-1"  # Clicked

# Step 3: Testing Loop
print("Starting Testing...")

# Add new columns to the DataFrame
behavior_df['Clicked News IDs'] = ""
behavior_df['Not-Clicked News IDs'] = ""
behavior_df['1 & 0 Predictions of Displayed News List'] = ""

# Process in batches for efficiency
for batch_start in tqdm(range(0, len(behavior_df), BATCH_SIZE)):
    batch_df = behavior_df.iloc[batch_start:batch_start + BATCH_SIZE]

    # Skip empty batches
    if batch_df.empty:
        continue

    # Prepare results for this batch
    def process_row(row):
        """Process a single row in the batch."""
        user_id = row['User ID']
        displayed_news_ids = row['Displayed News List']

        # Ensure valid list
        displayed_news_ids = displayed_news_ids or []
        displayed_news_ids = [nid for nid in displayed_news_ids if nid in news_embeddings_map]

        # Skip if no valid news IDs
        if not displayed_news_ids:
            return "", "", ""

        user_profile = user_profiles.get(user_id, default_user_profile)
        preference_profile = user_profile['preference_profile']
        non_preference_profile = user_profile['non_preference_profile']

        clicked_news = []
        not_clicked_news = []
        predictions = []

        with ThreadPoolExecutor() as executor:
            results = list(executor.map(
                lambda news_id: classify_news_parallel(news_id, preference_profile, non_preference_profile, count_tracker),
                displayed_news_ids
            ))

        for classification, prediction in results:
            if classification is None:
                continue

            # Append to appropriate list based on the classification
            if prediction.endswith("-0"):
                not_clicked_news.append(classification)
            else:
                clicked_news.append(classification)

            # Append prediction to the predictions list
            predictions.append(prediction)

        return ','.join(clicked_news), ','.join(not_clicked_news), ','.join(predictions)

    # Process each row in the batch
    batch_results = batch_df.apply(process_row, axis=1)
    batch_df['Clicked News IDs'], batch_df['Not-Clicked News IDs'], batch_df['1 & 0 Predictions of Displayed News List'] = zip(*batch_results)

    # Update results back to the main DataFrame
    behavior_df.update(batch_df)

# Step 4: Save results to shared path
print("Saving results to the specified path...")
behavior_df.to_csv(output_csv_path, index=False)
print(f"Results successfully saved to: {output_csv_path}")

# Step 5: Print results
print(f"Testing completed.")
print(f"News IDs Found: {count_tracker['news_ids_found']}")
print(f"News IDs Not Found: {count_tracker['news_ids_not_found']}")
print("Testing predictions have been written to the behavior DataFrame.")

Testing data loaded successfully during the data processing steps.
Starting Testing...


100%|██████████████████████████████████████| 235/235 [9:39:01<00:00, 147.83s/it]


Saving results to the specified path...
Results successfully saved to: /Users/n7/Desktop/ie University SAMBD Acadamics/Capstone Project Revised/Data/MINDlarge_test/Behavior Predictions Test Results/behavior_predictions_results.csv
Testing completed.
News IDs Found: 95327863
News IDs Not Found: 0
Testing predictions have been written to the behavior DataFrame.
